# About

This notebook will introduce the `LIMIT`, `SKIP`, and `UNION cypher keywords.

In [1]:
from neo4j import GraphDatabase, Record, ResultSummary, EagerResult
from neo4j.time import Date

import pandas as pd
pd.set_option('display.max_colwidth', 100)

import os 
import sys
import socket
from dotenv import load_dotenv 
load_dotenv()

# Add the utils directory to sys.path
sys.path.append(os.path.abspath("../utils"))

from Neo4jParser import Neo4jParser


NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## `LIMIT`

* `LIMIT` will contrain the number of records returned to the user.
* For example, `MATCH (p:Person) RETURN p.name LIMIT 25;` will only return the first 25 records.
* `LIMIT` can also except expressions, as long as the result is an integer.

In [8]:
# Let's find all nodes in the graph
result = driver.execute_query(
    """ 
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 139 records after 6 ms and completed after 8 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name;
    


{'p.name': ['Aaron Sorkin',
  'Al Pacino',
  'Andy Wachowski',
  'Angela Scope',
  'Annabella Sciorra',
  'Anthony Edwards',
  'Audrey Tautou',
  'Ben Miles',
  'Bill Paxton',
  'Bill Pullman',
  'Billy Crystal',
  'Bonnie Hunt',
  'Brooke Langton',
  'Bruno Kirby',
  'Cameron Crowe',
  'Carrie Fisher',
  'Carrie-Anne Moss',
  'Charlize Theron',
  'Chris Columbus',
  'Christian Bale',
  'Christina Ricci',
  'Christopher Guest',
  'Clint Eastwood',
  'Corey Feldman',
  'Cuba Gooding Jr.',
  'Danny DeVito',
  'Dave Chappelle',
  'David Morse',
  'Demi Moore',
  'Diane Keaton',
  'Dina Meyer',
  'Ed Harris',
  'Emil Eifrem',
  'Emile Hirsch',
  'Ethan Hawke',
  'Frank Darabont',
  'Frank Langella',
  'Gary Sinise',
  'Geena Davis',
  'Gene Hackman',
  'Greg Kinnear',
  'Halle Berry',
  'Helen Hunt',
  'Howard Deutch',
  'Hugo Weaving',
  'Ian McKellen',
  'Ice-T',
  'J.T. Walsh',
  'Jack Nicholson',
  'James Cromwell',
  'James L. Brooks',
  'James Marshall',
  'James Thompson',
  'Jan de

In [9]:
# Let's find all nodes in the graph, but limit to the first 25
result = driver.execute_query(
    """ 
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name LIMIT 5;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 5 records after 0 ms and completed after 0 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name LIMIT 5;
    


{'p.name': ['Aaron Sorkin',
  'Al Pacino',
  'Andy Wachowski',
  'Angela Scope',
  'Annabella Sciorra']}

In [11]:
# Let's find all nodes in the graph, but limit with an expression
result = driver.execute_query(
    """ 
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name LIMIT toInteger(floor(3.3));
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 3 records after 32 ms and completed after 32 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) 
    RETURN p.name ORDER BY p.name LIMIT toInteger(floor(3.3));
    


{'p.name': ['Aaron Sorkin', 'Al Pacino', 'Andy Wachowski']}

In [13]:
# Let's create a node but return with a limit of 0
result = driver.execute_query(
    """ 
    CREATE (n {id:"test123abc"}) RETURN n LIMIT 0;
    """,
    database_="dev"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 0 records after 22 ms and completed after 30 ms.

Query executed against database: 'dev':  
    CREATE (n {id:"test123abc"}) RETURN n LIMIT 0;
    


{}

**NOTE:** What the query above is saying, is create a new node n, but I don't want to see it. Just create it. Now we can run a query below to match for it and find what we just created.

In [14]:
# Let's create a node but return with a limit of 0
result = driver.execute_query(
    """ 
    MATCH (n {id:"test123abc"}) RETURN n;
    """,
    database_="dev"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 12 ms and completed after 13 ms.

Query executed against database: 'dev':  
    MATCH (n {id:"test123abc"}) RETURN n;
    


{'n': [{'elementId': '4:ff73e06d-56ad-4959-b409-fcc3d9dce978:1',
   'labels': frozenset(),
   'properties': {'id': 'test123abc'}}]}

## `SKIP`

* `SKIP` will tell Neo4j the row to start from. Where `LIMIT` returns the first *x* records, `SKIP` tells Neo4j to ignore the first *x* records.
* `SKIP` and limit can be combined to 'skip' the first *x* records and then return the next *y* through `LIMIT`.
    * **IMPORTANT** To avoid a syntax error, `SKIP` comes before `LIMIT` after the `RETURN` clause.
* Just like `LIMIT`, `SKIP` can also accept expressions as long as the result of the expression is an integer.

In [21]:
# Let's return the names of people in alphabeutical order, but skip the first 100 names
result = driver.execute_query(
    """ 
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 100;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 39 records after 0 ms and completed after 1 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 100;
    


{'p.name': ['Penny Marshall',
  'Philip Seymour Hoffman',
  'Rain',
  'Regina King',
  'Renee Zellweger',
  'Richard Harris',
  'Rick Yune',
  'Rita Wilson',
  'River Phoenix',
  'Rob Reiner',
  'Robert De Niro',
  'Robert Longo',
  'Robert Zemeckis',
  'Robin Williams',
  'Ron Howard',
  "Rosie O'Donnell",
  'Sam Rockwell',
  'Scott Hicks',
  'So-dam Park',
  'Stephen Rea',
  'Steve Zahn',
  'Sun-kyun Lee',
  'Susan Sarandon',
  'Takeshi Kitano',
  'Taylor Hackford',
  'Tom Cruise',
  'Tom Hanks',
  'Tom Skerritt',
  'Tom Tykwer',
  'Tony Scott',
  'Val Kilmer',
  'Victor Garber',
  'Vincent Ward',
  'Werner Herzog',
  'Wil Wheaton',
  'Woo-sik Choi',
  'Yeo-jeong Jo',
  'Zach Grenier',
  'Zazie Beetz']}

In [22]:
# Let's return the first 5 names of people in alphabeutical order, after skipping the first 100 names
result = driver.execute_query(
    """ 
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 100 LIMIT 5;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 5 records after 0 ms and completed after 2 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 100 LIMIT 5;
    


{'p.name': ['Penny Marshall',
  'Philip Seymour Hoffman',
  'Rain',
  'Regina King',
  'Renee Zellweger']}

In [23]:
# Let's return the first 5 names of people in alphabeutical order, after skipping the first 100 names
result = driver.execute_query(
    """ 
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 10*10 LIMIT 2+3;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 5 records after 18 ms and completed after 18 ms.

Query executed against database: 'neo4j':  
    MATCH (p:Person) RETURN p.name ORDER BY p.name SKIP 10*10 LIMIT 2+3;
    


{'p.name': ['Penny Marshall',
  'Philip Seymour Hoffman',
  'Rain',
  'Regina King',
  'Renee Zellweger']}

## `UNION`

* `UNION` can be used to combine the results of multiple queries, removing duplicates. Works very similar to SQL's `UNION` and `UNION ALL`.
* When returning results in a query with `UNION`, the results need to have the same alias. 
    * For example: <br>
        `MATCH (m:Movie)`<br>
        `RETURN m.title AS custom_field LIMIT 2`<br>
        `UNION`<br>
        `MATCH (p:Person)`<br>
        `RETURN p.name AS custom_field LIMIT 2;`<br>
* `UNION ALL` will return all records, including duplicates.

In [25]:
# Combine movie titles and people names in the same result
result = driver.execute_query(
    """ 
    MATCH (m:Movie)
    RETURN m.title AS custom_field LIMIT 2
    UNION
    MATCH (p:Person) 
    RETURN p.name AS custom_field LIMIT 2;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 4 records after 66 ms and completed after 66 ms.

Query executed against database: 'neo4j':  
    MATCH (m:Movie)
    RETURN m.title AS custom_field LIMIT 2
    UNION
    MATCH (p:Person) 
    RETURN p.name AS custom_field LIMIT 2;
    


{'custom_field': ['The Matrix',
  'The Matrix Reloaded',
  'Keanu Reeves',
  'Carrie-Anne Moss']}

In [26]:
# Combine movie titles and people names in the same result, keeping all records
result = driver.execute_query(
    """ 
    MATCH (m:Movie)
    RETURN m.title AS custom_field LIMIT 2
    UNION ALL
    MATCH (p:Person) 
    RETURN p.name AS custom_field LIMIT 2;
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 4 records after 52 ms and completed after 52 ms.

Query executed against database: 'neo4j':  
    MATCH (m:Movie)
    RETURN m.title AS custom_field LIMIT 2
    UNION ALL
    MATCH (p:Person) 
    RETURN p.name AS custom_field LIMIT 2;
    


{'custom_field': ['The Matrix',
  'The Matrix Reloaded',
  'Keanu Reeves',
  'Carrie-Anne Moss']}